# ChromeDevToolで計測する

- 2018/10/3 勉強会


# アジェンダ

1. 大まかなレンダリングの流れ
2. RAILというパフォーマンスの指標
3. Chrome Developer Toolで計測

# 大まかなレンダリングの流れ

- ChromeのPerformanceのページの円グラフの項目の通り
- この一連の処理を「クリティカルレンダリングパス」と呼ぶ

![](https://github.com/mrsekut/slides/blob/master/img/rendering_flow.png?raw=true)

## ざっくり概説

### Loading

- DNSの名前解決、TCPの接続確立、HTTPによる通信
- プログラム、画像などのリソースの読み込み
- DOM,CSSOMツリーの作成

### Scripting
- JavaScriptの字句解析、構文解析
- コンパイル
- 実行

### Rendering
- Styleの計算
- RenderTreeの作成

### Painting
- レイヤーごとにピクセル描画
- レイヤーの合成

# パーフォマンスの指標

- RAIL
    - 「ハイパフォーマンスブラウザネットワーキング」の著者のGoogleのおじさんが提唱
- Response: 100ms
- Animation: 16ms
- Idle: 50ms
- Load: 1,000ms

------
- 【参考】[RAIL モデルでパフォーマンスを計測する  |  Web  |  Google Developers](https://developers.google.com/web/fundamentals/performance/rail?hl=ja)

# Chrome Developer Toolで計測する


- Network
- Performance
- Memory
- Rendering
- Audits

とりあえず、`cmd-alt-i` で開いてみましょう



## Networkパネル

- リソース取得のタイムラインとリソースの詳細
----
- `Disable cache`にチェックでキャッシュなしでリロード
- `Offline`にチェックでネット切断
----
- 青い線はDOMContentLoadedイベントの発生点
    - これが遅いとスクリプトによる読み込みブロックが起きている
- 赤い線はLoadイベントの発生点
    - これが遅いとリソースが大きすぎる


### 詳細のTimingを見てみる

- 適当にリソースを選択すると、右側で詳細が見れる
- 今回見るのは「Timing」の欄
![](https://github.com/mrsekut/slides/blob/master/img/devtool_timing.png?raw=true)

#### Resource Scheduling

- HTTPよりも低レイヤであるTCP, TLSなどの接続確立の時間など

#### Request/Response

- Waiting(TTFB)
    - リクエスト送信後から、最初のデータを受信するまでの時間
    - これが長いと、サーバーでの処理に時間がかかっている
    - [200ms以下が望ましい](https://developers.google.com/speed/docs/insights/Server)

### 実践

- imgにチェックしてみるとか
- `size`でソートしてみるとか

- ログイン済みのときもアプリ詳細の画像を取ってきてる

### Network
    
### TTFBを減らす

- 速度の遅いアプリケーションロジック
- 遅いデータベースクエリ
- 遅いルーティング
- フレームワーク
- ライブラリ
- リソースによるCPU の消費
- メモリ不足

- PageSpeed Insightsのルールより。
    - https://developers.google.com/speed/docs/insights/Server

## Performanceパネル

- ページの読み込み、操作に関する全てのイベントを分析
- 左上の丸マークかリロードマークで計測
- ボトルネックににっているものには右上赤三角が表示されている
- 【参考】[中部チャートの項目の概説](https://developers.google.com/web/tools/chrome-devtools/evaluate-performance/performance-reference)
---
- 画面上部の緑のグラフはFPS
    - 頂上が60FPS
    - その上部が赤くなっているところは、60FPSを切ったフレーム(フレーム落ち)
    - 赤がなく、緑の面積が大きいほどいい
    - [サイボウズのブログ 計測とその具体的な対処法が紹介されている](https://myakura.hatenablog.com/entry/2016/05/12/083000)
----
- Bottole-Upなどに重い処理の呼び出し元の関数名も表示されている
    - webpackのせいかわかりにくい

### GC

- `cmd-f`などで「GC」と検索すればタイミングを見れる
- アニメーション時などに起きると画面がもたつくので対策が必要
- 左上のゴミ箱マークをクリックするとGCを発動

### 実践

- 右上赤三角を見てみよう
- メモリのグラフを見てGCが起きている部分を見てみよう
    - 「GC」と検索すればその前後に見つかるはず
- 重い処理
    - 通知のページ
    - 投稿のアイテム追加の検索のページ
    - my pageに来たとき


### Performanceパネル
- ページの読み込み、操作に関するすべてのイベントを分析

- ドキュメント
    - https://developers.google.com/web/tools/chrome-devtools/evaluate-performance/performance-reference
    
- 中部のやつのことを「フレームチャート」、下部のことを「詳細パネル」と呼ぶ

- ボトルネックになっているものにはフレームチャート上の右上に赤くなって示してくれている

- 黄色がJavaScript


#### 詳細ペインのタブ
- summaryタブ
    - 処理の実行時間、関数名など
- bottom-upタブ
    - パフォーマンスに影響を与えている関数とそれがどこから呼び出されれているかなどの情報
- Call Tree
    - 関数の呼び出し元からツリー状に呼び出しの全体像をたどる
    - Reactではむずい？
- Event Log
    - 開始からの処理のログ


#### GC

- cmd-fで「GC」で検索するとGCしてる部分が確認できる

- Minor GC
- Major GC
- DOM GC
などが確認できた

- アニメーション処理中にGCが起こると描画がもたつくので避けるべき
    - アニメーション時に新たなオブジェクトを生成しない
    - もしくはアニメーション処理より前にオブジェクトを確保し、処理中は計算のみ行わせる(確保したときにGCを起こるなら起こさせる)
    
- GCの原因を探る
    - 録画して検索してGCの場所を特定
    - そのGCの引き金になった関数などを特定
    - その関数部分をコメントアウトし、再実行
    - GCが行われないかを確認する
    
- Performanceタブの左上のゴミ箱マーククリックでGCできる
    - 録画中の最後の方にクリックするとGC発動していることが確認できる
    
#### メモリリーク

- メモリが開放されずに使わないのにメモリを確保し続けること

- MMU
- メモリの保護
    - 他のプロセスのメモリを読まない
- メモリスワッピング
    - 仮想メモリを補助記憶装置も使う
    - そこにアクセスするときは補助記憶装置からメモリに復帰させてから読むので遅くなる(スワップイン、スワップアウト)
    - モバイルOSだとメモリスワッピングが弱いのでクラッシュする可能性もある
    
- console.logなどはGCの対象にならないのでメモリリークを引き起こす可能性がある
#### Memoryにチェックをつける

- 正常な場合、鋸歯のようになる Wp159の図参照



##### 線
それぞれ何？  

- JS Heap
- Documents
- Nodes
    - DOM数のこと(?)
- Listeners
 - GPU Memory

## Memoryパネル

- メモリの利用状況
- メモリリークの検知やメモリの使いすぎを把握
- 「Record Allocation Timeline」でメモリリークが起きている箇所を見つける

### Record Allocation Timeline

- 50ms毎にヒープのスナップショットを取得し、記録終了時に最後のスナップショットを取得
----
- 縦線の高さ: オブジェクトのサイズ
- 青: タイムラインの終了時まで存在しているオブジェクト
- 灰: タイムライン中にGCが行われたオブジェクト
----
- @の数字はオブジェクトID
----
- GCが行われなかったということは、どこかで参照されているから


### Memory

- heap snapshot
    - JSのオブジェクトとそれに付随したDOMノード間のメモリ配分を見る
    - https://developers.google.com/web/tools/chrome-devtools/memory-problems/heap-snapshots?hl=ja
    - [三点観測](http://www.techscore.com/blog/2017/12/24/%E3%83%95%E3%83%AD%E3%83%B3%E3%83%88%E3%82%A8%E3%83%B3%E3%83%89%E3%81%AE%E3%83%91%E3%83%95%E3%82%A9%E3%83%BC%E3%83%9E%E3%83%B3%E3%82%B9%E6%94%B9%E5%96%84%E3%81%A8%E3%83%A1%E3%83%A2%E3%83%AA%E3%83%AA/)
- record allocation profile
    - js各関数のメモリの割り当てを見る
- record allocation timeline
    - メモリリークがどこで起きているのかを見る
    - 50ms毎にヒープのスナップショットを取得し、記録終了時に最後のスナップショットを取得
    - [参考](https://developers.google.com/web/tools/chrome-devtools/memory-problems/allocation-profiler?hl=ja)
    - @の数字はオブジェクトID
    - 縦線の高さ: オブジェクトのサイズ
    - 青: タイムラインの終了時まで存在しているオブジェクト
    - 灰: タイムライン中にGCが行われたオブジェクト
    - GCが行われなかったということは、どこかで参照されているから
    


### Rendering

- 右上の点が縦に３つ並ぶメニューから「Rendering」を選択
- 「Paint flashing」にチェックを付けると、再レンダリングされたところが緑になる
- 【参考】[Chrome DevTools の「Enable paint flashing」を使う - kitak's blog](https://kitak.hatenablog.jp/entry/2015/11/01/152102)



### 実践

- 「Paint flashing」にチェックを付けよう
- スクロールやページ遷移をして無駄な再描画がないか確認しよう

### Rendering
#### その他の項目
- Paint Flashing
- Layer borders
    - Compositing Layerとtileを表示する
    - https://www.html5rocks.com/ja/tutorials/speed/layers/
- FPS Meter
    - 現在のFrame RateとGPUラスタライズの有効状況、GPUメモリの状況を表示する
- Scrolling Perfomrmance Issues
    - スクロールのレスポンスを阻害するようなEvent Listenerやスクロールの度に再描画される要素があれば画面内に表示する
- Emulate CSS Media
    - CSSのMediaタイプをprint/screenと切り替える



https://qiita.com/damele0n/items/363d6e30b92ed0c75b02#%E9%A0%85%E7%9B%AE

### Auditsパネル

- パフォーマンス診断ツール
    - PageSpeed Insightsみたいなやつ
- カテゴリー別の課題とその深刻度を表示
- クリックするとその解決法を教えてくれる


# 所感

- これまで気にしてなかったが調べてみたら関連記事が山ほど出てきた
- 問題は見つけられても解決できなければ意味がない
- Reactの場合
    - Reactの内部構造を知る必要がある
        - 差分描画の仕組みとか
        - やるしかない
    - ReactはReactで便利な解析ツールがありそう
        - react-perf-devtoolとか

## chrome://tracing

- 公式ドキュメント
    - https://www.chromium.org/developers/how-tos/trace-event-profiling-tool

## 他にもJS APIがいっぱいある

- パフォーマンス計測

##  パフォーマンス診断ツール
- PageSpeed Insightsなど

## 継続的パーフォマンス監視

- New Relic Browser

# 参考

- [dev tool 公式](https://developers.google.com/web/tools/chrome-devtools/)
- https://webtan.impress.co.jp/e/2017/02/20/24816
- https://qiita.com/y_fujieda/items/a0a69151cf7307039f74
- https://www.slideshare.net/yoshikawa_t/chrome-developer-tools-17787728

# 関連(もっかい読む)
- [dev toolを使ったReactのチューニング AbemaTV](https://1000ch.net/posts/2016/abematv-runtime-perf-audit.html)